In [ ]:
import pickle
import pandas as pd
import gc
import numpy as np
import warnings
import pyarrow.parquet as pq

warnings.filterwarnings('ignore')

def r2_metric(y_true, y_pred, weights=None):
    """Calculate weighted R2 score"""
    y_true = y_true.ravel()
    y_pred = y_pred.ravel()

    # If weights is None, use uniform weights
    if weights is None:
        weights = np.ones_like(y_true)
    else:
        weights = weights.ravel()

    numerator = np.sum(weights * (y_true - y_pred) ** 2)
    denominator = np.sum(weights * (y_true ** 2))
    r2_score = 1 - (numerator / denominator)
    return 'r2', r2_score, True

### Selected Rolling Model 
- Partition6 | 1_000_000~5_000_000 학습
- 5_000_000 ~ 6_000_000 Inference

- 각 symbol 별로 Moving Average를 진행함. 
- Feature는 대표 feature만 추가해서 진행 

In [ ]:
selected_rolling_model_path = "experiment_models/selected_feature_rolling_model_1.pkl"

with open(selected_rolling_model_path, 'rb') as f:
    selected_rolling_model = pickle.load(f)
    
columns = pq.read_schema(f'eda_chunks/final_rolling_{str(2).zfill(4)}.parquet').names
columns = pd.Index(columns)  # pandas Index로 변환하여 str.contains 사용 가능하게 함

# 필요한 컬럼을 선택합니다
not_rolling_feature_col = columns[
        (columns.str.contains("feature")) & 
        (~columns.str.contains("rolling"))
    ].to_list()

rolling_feature_col = columns[
        (columns.str.contains("rolling")) & 
        (~columns.str.contains("rolling_2")) & 
        (~columns.str.contains("rolling_30")) & 
        (~columns.str.contains("rolling_1000"))
    ].to_list()

selected_columns = not_rolling_feature_col + rolling_feature_col + ['responder_6'] + ['weight']
selected_rolling_df = pd.read_parquet(f'eda_chunks/final_rolling_{str(2).zfill(4)}.parquet', columns=selected_columns)


selected_rolling_target_col = "responder_6"
selected_rolling_weights = selected_rolling_df['weight']
selected_rolling_X = selected_rolling_df[not_rolling_feature_col + rolling_feature_col]
selected_rolling_y = selected_rolling_df[selected_rolling_target_col]

In [ ]:
selected_rolling_model

LGBMRegressor(random_state=42, verbose=0)

In [ ]:
selected_rolling_pred = selected_rolling_model.predict(selected_rolling_X)
r2_metric(selected_rolling_y, selected_rolling_pred, selected_rolling_weights)

: 

In [ ]:
selected_rolling_pred

array([  6.31454886,   5.10559914, -13.52438586, ...,   3.05871051,
        -2.7365602 ,  -4.93815975])

#### lgbm model __dict__